## Data analysis
https://www.r-graph-gallery.com/101_Manhattan_plot.html

## Running sos notebook

On the Yale Farnam cluster,


On a local computer with for example 8 threads,

```
sos run BOLT-LMM_R_analysis_UKB.ipynb -q none -j 8
```


In [85]:
[global]
# Working directory
parameter: cwd = path('~/pleiotropy_UKB/data')
#Input individuals summary stats file
parameter: input_pattern = 'chr?.gz'
import glob
input_file = sorted(glob.glob(input_pattern))
fail_if(len(input_file) == 0, msg = f"Input pattern ``{input_pattern}`` failed to match any files.")
#Output merged summary stats file
parameter: output_file = 'summary_stats.txt'
#Ouput Manhattan plot
parameter: manhattan= 'INT-BMI_manhattan'

## Merging summary stats for all chromosomes

In [86]:
[step1]
input: input_file
output: f'{output_file}.gz'
python: expand=True
import gzip
with gzip.open ('{_output}', 'wt') as outfile:
    with gzip.open({_input[0]:r}) as f:
        for line in f:
            outfile.write(line.decode('utf-8'))
        for files in [{_input:r,}][1:]:
            with gzip.open(files) as f:
                for line in f:
                    if not line.startswith(b'SNP'):
                        outfile.write(line.decode('utf-8'))

In [ ]:
sum(1 for i in open('test.txt', 'r'))

## Creating the Q-Q plot and Manhattan plots

In [84]:
[step2]
input: output_file
output: manhattan
bash: expand = "${ }", workdir = cwd
R: expand=True
    install.packages("qqman")
    library(qqman)
    summary_gz=gzfile('{_name}.gz','rt')
    df=read.table('{_name}', header=TRUE,sep = ' ')
    png('{_manhattan}.png', width = 6, height = 4, unit='in', res=300)
    bmi_manhattan <- manhattan(df, chr='CHR', bp='BP', snp='SNP', p='P')
    dev.off()



Install and load required libraries

In [ ]:
#install.packages("reader")
#intall.packages("plyr")
#install.packages("ggplot2")
#library(readr)
#library(plyr)
#library(ggplot2)

Then create a list of files to be combined using R. In this case the summary statistics for the association analysis and then create the dataset by binding files from chr{1:22}. The drawback is that R runs out of memory when doing this

In [ ]:
#file_list = Sys.glob("*.snp_stats.bgen.gz") #one way of creating lists
#file_list = list.files(path=mydir, pattern="*.snp_stats.bgen.gz", full.names=TRUE) #another way of creating lists
#data.list = lapply(file_list, function(x){read.table(file = x,header = TRUE, sep = "\t")})
#data.merged = do.call("rbind", data.list

## Drawing the Manhattan Plot

In [ ]:

# Let's highlight them, with a bit of customization on the plot
#manhattan(gwasResults, highlight = snpsOfInterest)
#Annotate the SNPs of interest
#manhattan(gwasResults, annotatePval = 0.01)
#Make a qqplot
#qq(gwasResults$P)